In [17]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

In [30]:
folder = "assets/hpc_out/5_1stm_oj_0_0_os_0_85/"
#folder = "assets/hpc_out/2_6r7m_oj_0_0_os_0_85/"
folder = "assets/hpc_out/"

max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, "_")[3]) ? parse(Int, split(file, "_")[3]) : max
    end
end

evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        @load "$folder$file" in_out_data
        id = parse(Int, split(file, "_")[3])
        string = "$(id): $(in_out_data.input.T) | $(in_out_data.input.ε) | $(in_out_data.output.αs[end]) | $(in_out_data.output.Es[1]), $(minimum(in_out_data.output.Es)), $(in_out_data.output.Es[end])"
        evaluation_strings[id] = string
    end
end

In [31]:
for elem in evaluation_strings
    println(elem)
end

1: 2.5 | 0.0 | 0.48538646 | 18630.973738632594, 18262.52572055894, 18286.206295301
2: 2.5 | 0.0 | 0.18899584 | 18379.67856719139, 18243.064132639873, 18305.342114054714
3: 2.5 | 0.0 | 0.4862321 | 18379.678566596005, 18316.95062034018, 18330.122710162646
4: 2.5 | 0.0 | 0.18660685 | 18378.681987460513, 18259.617536746762, 18287.317570640498



8: 2.5 | 0.0 | 0.46579418 | 18379.678566890525, 18303.02472357062, 18319.043122889798

10: 2.5 | 0.0 | 0.27048185 | 18379.678567011953, 18207.707767716824, 18216.006802702123
11: 2.5 | 0.0 | 0.4021813 | 18379.67856710009, 18288.41845382677, 18326.324624017674
12: 2.5 | 0.0 | 0.38505468 | 18374.877573090376, 18308.3372398765, 18343.79593876261
13: 2.5 | 0.0 | 0.2070121 | 18371.44888036818, 18232.354739243165, 18262.01237576966
14: 2.5 | 0.0 | 0.1730955 | 18373.297245209727, 18259.459912427636, 18276.99566847363
15: 2.5 | 0.0 | 0.19933425 | 18379.678567005252, 18262.945480262824, 18285.464088795994
16: 2.5 | 0.0 | 0.33678034 | 18723.230043517815, 182

In [32]:
# folder = "assets/hpc_out/5_1stm_oj_0_0_os_0_85/"
# folder = "assets/hpc_out/2_6r7m_oj_0_0_os_0_85/"

for file in readdir(folder)
    try
        @load "$folder$file" in_out_data
        mindex = argmin(in_out_data.output.Es)
        state = in_out_data.output.states[mindex]
        id = parse(Int, split(file, "_")[3])
        radii = collect(Base.Iterators.flatten([in_out_data.input.template_radii for _ in 1:in_out_data.input.number_of_molecules]))
        MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, in_out_data.input.template_mol), radii, "$(folder)$(id)", in_out_data.input.number_of_molecules, length(in_out_data.input.template_radii))
    catch e
        println(e)
        continue
    end
end

In [10]:
@load "assets/hpc_assembly/3_6r7m_assembled_oj_0_0_os_0_85/49_6r7m_protor_oj_0_0_os_0_85.jld2" in_out_data
state_sim = in_out_data.output.states[argmin(in_out_data.output.Es)]
template_mol_sim = in_out_data.input.template_mol

template_mol_exp, _ = MorphoMol.Utilities.poly_to_state("assets/input/poly/6r7m_protor.poly")
template_mol_exp = reshape(template_mol_exp,(3,Int(length(template_mol_exp) / 3)))

R1 = log(RotMatrix(@SMatrix[0.628642  0.777695  0.000000; -0.777695  0.628642  0.000000; 0.000000  0.000000  1.000000]))
T1 = @SVector[-69.28043, 195.91352, -49.35000]

R2 = log(RotMatrix(@SMatrix[0.874450  0.485115  0.000000; -0.485115  0.874450  0.000000; 0.000000  0.000000  1.000000]))
T2 = @SVector[-61.30589, 104.11829, -47.94000]

R3 = log(RotMatrix(@SMatrix[0.992567  0.121696  0.000000; -0.121696  0.992567  0.000000; 0.000000  0.000000  1.000000]))
T3 = @SVector[-19.48193, 22.01644, -46.53000]

state_exp_assembled = Vector{Float64}([])
for (R,T) in [(R1, T1), (R2, T2), (R3, T3)]
    state_exp_assembled = [state_exp_assembled; [R[3,2], R[1,3], R[2,1], T[1], T[2], T[3]]]
end

permutations = [[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3,1,2], [3,2,1]]
for perm in permutations
    println(MorphoMol.Utilities.sum_of_permutation(template_mol_exp, template_mol_sim, state_exp_assembled, state_sim, [1, 2, 3], perm))
end

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 ~/.julia/packages/JLD2/twZ5D/src/JLD2.jl:298


LoadError: SystemError: opening file "assets/hpc_assembly/3_6r7m_assembled_oj_0_0_os_0_85/49_6r7m_protor_oj_0_0_os_0_85.jld2": No such file or directory

In [58]:
@load "assets/hpc_assembly/3_6r7m_stacked_oj_0_0_os_0_85/43_6r7m_protor_oj_0_0_os_0_85.jld2" in_out_data
state_sim = in_out_data.output.states[argmin(in_out_data.output.Es)]
template_mol_sim = in_out_data.input.template_mol

template_mol_exp, _ = MorphoMol.Utilities.poly_to_state("assets/input/poly/6r7m_protor.poly")
template_mol_exp = reshape(template_mol_exp,(3,Int(length(template_mol_exp) / 3)))

R1 = log(RotMatrix(@SMatrix[0.628642  0.777695  0.000000; -0.777695  0.628642  0.000000; 0.000000  0.000000  1.000000]))
T1 = @SVector[-69.28043, 195.91352, -49.35000]

R2 = log(RotMatrix(@SMatrix[0.874450  0.485115  0.000000; -0.485115  0.874450  0.000000; 0.000000  0.000000  1.000000]))
T2 = @SVector[-61.30589, 104.11829, -47.94000]

R3 = log(RotMatrix(@SMatrix[0.803441  0.595384  0.000000; -0.595384  0.803441  0.000000; 0.000000  0.000000  1.000000]))
T3 = @SVector[-67.99970, 135.02619, -25.38000]


state_exp_assembled = Vector{Float64}([])
for (R,T) in [(R1, T1), (R2, T2), (R3, T3)]
    state_exp_assembled = [state_exp_assembled; [R[3,2], R[1,3], R[2,1], T[1], T[2], T[3]]]
end

permutations = [[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3,1,2], [3,2,1]]
for perm in permutations
    println(sum_of_permutation(template_mol_exp, template_mol_sim, state_exp_assembled, state_sim, [1, 2, 3], perm))
    println()
end

1.060546051243098, 1.7233861453834445, 2.704512138639089
1.829481445088544

3.8582220739511452, 1.7233861453834445, 7.206604684868613
4.262737634734401

1.060546051243098, 1.9544367320546299, 6.025618439072101
3.0135337407899434

5.358151778229962, 1.9544367320546299, 7.206604684868613
4.839731065051068

3.8582220739511452, 5.183991323542944, 6.025618439072101
5.02261061218873

5.358151778229962, 5.183991323542944, 2.704512138639089
4.415551746803998

